# 1. Inference

In [1]:
import os
import sys

import numpy as np
import tensorflow as tf

from deepcell.applications import NuclearSegmentation, CellTracking
from deepcell_tracking.trk_io import load_trks

sys.path.append('/publication-tracking/benchmarking')
import utils

In [2]:
source_data = '/publication-tracking/data/test.trks'

data_dir = '/publication-tracking/benchmarking/DeepCell/data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

for d in [data_dir, gt_seg_dir, pred_seg_dir]:
    if not os.path.exists(d):
        os.makedirs(d)
        
model_urls = {
    'NuclearSegmentation': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearSegmentation-7.tar.gz',
    'NuclearTrackingNE': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearTrackingNE-7.tar.gz',
    'NuclearTrackingInf': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearTrackingInf-7.tar.gz'
}

In [3]:
# Load test data
data = load_trks(source_data)

In [4]:
# Download and load each model
models = {}
for m, url in model_urls.items():
    archive_path = tf.keras.utils.get_file(f'{m}.tgz', url, extract=True, cache_subdir='models')
    model_path = os.path.splitext(archive_path)[0]
    model = tf.keras.models.load_model(model_path)
    models[m] = model

2022-11-17 23:46:52.323004: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 23:46:53.109959: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1


In [5]:
# Load segmentation and tracking applications
app_seg = NuclearSegmentation(models['NuclearSegmentation'])
app_trk = CellTracking(models['NuclearTrackingInf'], models['NuclearTrackingNE'])

In [6]:
for batch_no in range(len(data['lineages'])):
    print(batch_no)
    
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]
    
    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]
    
    # Generate tracks on GT segmentations
    track_gt = app_trk.track(X, y)
    track_gt['y_tracked'], track_gt['tracks'] = utils.convert_to_contiguous(track_gt['y_tracked'], track_gt['tracks'])
    utils.save_ctc_res(gt_seg_dir, batch_no + 1, track_gt['y_tracked'][..., 0], track_gt['tracks'])
    utils.save_ctc_gt(gt_seg_dir, batch_no + 1, y[..., 0], lineage)
    
    # Generate tracks on predicted segmentations
    y_pred = app_seg.predict(y)
    track_pred = app_trk.track(X, y_pred)
    track_pred['y_tracked'], track_pred['tracks'] = utils.convert_to_contiguous(track_pred['y_tracked'], track_pred['tracks'])
    utils.save_ctc_res(pred_seg_dir, batch_no + 1, track_pred['y_tracked'][..., 0], track_pred['tracks'])
    utils.save_ctc_gt(pred_seg_dir, batch_no + 1, y[..., 0], lineage)

0


2022-11-17 23:48:19.131720: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


1


/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


2


...

/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


10


/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


11


/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,


# 2. Evaluation

In [7]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [8]:
data_dir = '/publication-tracking/benchmarking/DeepCell/data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

pattern = re.compile('\d{3}_GT')
data_ids = [f.split('_')[0] for f in os.listdir(gt_seg_dir) if pattern.fullmatch(f)]

node_match_threshold = 0.6

ctc_software = '/publication-tracking/benchmarking/CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [9]:
benchmarks = []

for results_dir, s in zip([gt_seg_dir, pred_seg_dir], ['GT', 'Deepcell']):
    for data_id in data_ids:
        results = {
            'model': f'Deepcell - {s}',
            'data_id': os.path.splitext(data_id)[0]
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')
        
        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)
        
        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout
            
            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))
        
        benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

missed node 1_29 division completely
missed node 29_29 division completely
missed node 57_10 division completely
missed node 60_0 division completely
18_16 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 5_6 division completely
missed node 121_42 division completely
missed node 144_28 division completely
missed node 26_25 division completely
missed node 1_29 division completely
missed node 3_23 division completely
29_29 out degree = 1, daughters mismatch.
corrected division 3_23 as a frameshift division not an error
missed node 8_18 division completely
missed node 10_19 division completely
15_17 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 29_17 division completely
missed node 60_34 division completely
missed node 76_7 division completely
82_17 out degree = 2, daughters mismatch, gt and res degree equal.
83_18 out degree = 2, parents mismatch, gt and res degree equal.
corrected division 10_19 as a frameshift division not an error
cor

In [10]:
df

,model,data_id,correct_division,mismatch_division,false_positive_division,false_negative_division,total_divisions,aa_tp,aa_total,te_tp,te_total,DET,SEG,TRA
0,Deepcell - GT,003,6,0,0,2,8,2050,2055,2117,2122,1.000000,1.000000,0.999794
1,Deepcell - GT,005,2,0,0,0,2,690,690,712,712,1.000000,1.000000,1.000000
2,Deepcell - GT,012,16,0,0,0,16,9565,9796,9866,10097,1.000000,1.000000,0.999036
3,Deepcell - GT,007,1,0,0,0,1,195,199,202,206,1.000000,1.000000,0.999365
4,Deepcell - GT,008,18,0,0,2,20,4945,4945,5056,5056,1.000000,1.000000,0.999966
5,Deepcell - GT,010,53,1,0,0,54,9008,9090,9275,9357,1.000000,1.000000,0.999581
6,Deepcell - GT,011,15,0,0,0,15,3825,3833,3967,3975,1.000000,1.000000,0.999857
7,Deepcell - GT,002,1,0,0,0,1,1075,1075,1109,1109,1.000000,1.000000,1.000000
8,Deepcell - GT,004,1,0,0,0,1,521,521,540,540,1.000000,1.000000,1.000000
9,Deepcell - GT,009,54,0,3,3,57,14678,14806,15046,15174,1.000000,1.000000,0.999773
